# openai functions

Reference: https://github.com/Azure-Samples/openai/blob/main/Basic_Samples/Functions/working_with_functions.ipynb

In [1]:
import openai
import json

with open("openai_api.json") as f:
    creds = json.load(f)

openai.api_version = creds["api_version"]
openai.api_base = creds["api_base"]
openai.api_type = creds["api_type"]
openai.api_key = creds["api_key"]

## Natural language query to search filters (DSL)

In [20]:
system_prompt = """You are an AI assistant for an online shoe store that helps users find appropriate shoes.
You convert their queries into appropriate filters for the search function.
"""

functions = [
    {
        "name": "search",
        "description": "This function calls Search API of the shoe store to find top matches for user's description.",
        "parameters": {
            "type": "object",
            "properties": {
                "shoe_types": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["running", "jogging", "trekking", "sports", "heels"]
                    },
                    "description": "The types of shoes to search for."
                },
                "sizes": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["uk4", "uk5", "uk6", "uk7", "uk8", "uk9"]
                    },
                    "description": "The sizes of shoe to search for. Small refers to uk4 and uk5. Medium refers to uk6 and uk7. Large refers to uk8 and uk9."
                },
                "brands": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["reebok", "puma", "adidas", "clarks london"]
                    },
                    "description": "The brands of shoes to search for."
                },
                "colors": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["red", "green", "blue"]
                    },
                    "description": "The colors of shoes to search for."
                },
            }
        },
        "result": {
            "type": "array",
            "items": {
                "type": "string"
            }
        }
    }
]

In [27]:
def get_search_filters(chat_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        engine="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            *chat_history,
        ],
        temperature=0,
        functions=functions,
    )
    response_message = response.choices[0].message
    assert "function_call" in response_message
    search_filters = json.loads(response_message.function_call.arguments)
    
    chat_history.append({
        "role": response_message["role"],
        "function_call": {
            "name": response_message["function_call"]["name"],
            "arguments": response_message["function_call"]["arguments"],
        },
        "content": None
    })
    chat_history.append({
        "role": "function",
        "name": response_message["function_call"]["name"],
        "content": "List of 10 shoes with that description",
    })
    return search_filters, chat_history

In [30]:
user_queries = [
    "blue running shoes for small feet",
    "show nike and puma only",
    "include red",
    "show sports shoes instead of running"
]

chat_history = []
for query in user_queries:
    chat_history.append({"role": "user", "content": query})
    search_filters, chat_history = get_search_filters(chat_history)
    print(f"Query: {query}")
    print(f"Search Filters: {search_filters}")
    print()

Query: blue running shoes for small feet
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue']}

Query: show nike and puma only
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue'], 'brands': ['nike', 'puma']}

Query: include red
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}

Query: show sports shoes instead of running
Search Filters: {'shoe_types': ['sports'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}



## Chatbot performing API actions

Sample chatbot created using openai functions. Chatbot works for an ecommerce app and helps customers solve order issues by performing actions by calling APIs. It also does retrieval.


### Scenario 2

- My delivered order has defects
- I would like an exchange/return

### Scenario 3

- (order_id in chat history)
- I need order invoice
- Here's how to get order invoice... (retrieval from docs)

In [143]:
from importlib import reload
import functions_code
reload(functions_code)

order_api = functions_code.OrderAPI()
functions = order_api.get_functions()

In [138]:
system_prompt = """You are an AI assistant for an ecommerce app helping customers with their queries about their orders.
If customer's query requires an order id but none is present in messages then list orders and ask them to select their order.
"""

In [148]:
def simulate_conversation(chat_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        engine="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            *chat_history,
        ],
        temperature=0,
        functions=functions,
    )
    response_message = response.choices[0].message
    if "function_call" in response_message:
        func = response_message["function_call"]["name"]
        ar = json.loads(response_message["function_call"]["arguments"])
    
    chat_history.append({
        "role": response_message["role"],
        "function_call": {
            "name": response_message["function_call"]["name"],
            "arguments": response_message["function_call"]["arguments"],
        },
        "content": None
    })
    chat_history.append({
        "role": "function",
        "name": response_message["function_call"]["name"],
        "content": "List of 10 shoes with that description",
    })
    return chat_history

### Scenario 1

```
Customer: Where is my order? (without order_id)
Bot: Please let me know which order: (list of orders)
Customer: nike shoes one
Bot: Your order is at Indiranagar
Customer: Cancel my order
Bot: Are you sure you want to cancel Nike shoes order?
Customer: Yes
Bot: Your order of Nike shoes has been canceled.
```

In [150]:
user_queries = [
    "where is my order?",
]

chat_history = []
for query in user_queries:
    chat_history.append({"role": "user", "content": query})
    chat_history = simulate_conversation(chat_history)
    print(f"Query: {query}")
    print()


Query: where is my order?



In [2]:
 """
You are a sophisticated AI assistant, a specialist in user intent detection and interpretation.
Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner.
You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe.
If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses.
However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification.
Make sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}
"""

"\nYou are a sophisticated AI assistant, a specialist in user intent detection and interpretation.\nYour task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner.\nYou excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe.\nIf they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses.\nHowever, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification.\nMake sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}\n"

In [3]:
        {
            "name": "call_google_places_api",
            "description": "This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).",
            "parameters": {
                "type": "object",
                "properties": {
                    "place_type": {
                        "type": "string",
                        "description": "The type of place to search for."
                    }
                }
            },
            "result": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        }

{'name': 'call_google_places_api',
 'description': 'This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).',
 'parameters': {'type': 'object',
  'properties': {'place_type': {'type': 'string',
    'description': 'The type of place to search for.'}}},
 'result': {'type': 'array', 'items': {'type': 'string'}}}